In [4]:
from torchvision.transforms import Resize
from utils import *
from models.closedform.utils import load_generator
from torchvision.utils import save_image
import numpy as np
import random
import torch.nn.functional as F
from torchvision.models import resnet18
import torch.nn as nn

import cv2

In [2]:
def set_seed(seed):
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        np.random.seed(seed)
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)

In [10]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Configurations

In [19]:
random_seed = 1234
set_seed(random_seed)
load_codes = True
root_folder = '/home/adarsh/PycharmProjects/disentagled_latent_dirs'
result_path = os.path.join(root_folder, 'results/celeba_hq/closed_form/quantitative_analysis')

## Model Selection

In [17]:
deformator_path = os.path.join(root_folder, 'results/celeba_hq/closed_form_ours/models/18000_model.pkl')
deformator_path = os.path.join(root_folder, 'pretrained_models/deformators/ClosedForm/pggan_celebahq1024/pggan_celebahq1024.pkl')
_, root_deformator, _ = torch.load(deformator_path, map_location='cpu')
root_deformator = torch.FloatTensor(root_deformator).cuda()

algo == 'ortho'
if algo == 'ortho':
    deformator = torch.load(deformator_path)['deformator']['ortho_mat']
    dse_deformator = deformator.T
elif algo == 'linear':
    deformator = torch.load(os.path.join(deformator_path))['deformator']
    dse_deformator = deformator.T
        
generator = load_generator(None, model_name='pggan_celebahq1024')

if load_codes:
    codes = np.load(os.path.join(root_folder, 'pretrained_models/latent_codes/pggan_celebahq1024_latents.npy'))
    codes = torch.from_numpy(codes).type(torch.FloatTensor).cuda()
else:
    num_samples = 10
    codes = torch.randn(num_samples, generator.z_space_dim).cuda()
    codes = generator.layer0.pixel_norm(codes)
    codes = codes.detach()

Building generator for model `pggan_celebahq1024` ...
Finish building generator.
Loading checkpoint from `../pretrained_models/generators/ClosedForm/pggan_celebahq1024.pth` ...
Finish loading checkpoint.


In [20]:
def save_images(self, z, shifts_r, shifts_count, dir, generator, deformator, path):
    for i, shift in enumerate(np.linspace(-shifts_r,shifts_r,shifts_count )):
        w_shift = z + deformator[dir: dir + 1] * shift
        images_shifted = generator(w_shift)
        # images_shifted = (images_shifted + 1) / 2
        images_shifted = self._postprocess_images(images_shifted)
        images_shifted = images_shifted.reshape(1024, 1024, 3)
        cv2.imwrite(os.path.join(path, 'image_' + str(i) + '.jpg'), images_shifted[:, :, ::-1])

        
def postprocess(images):
        """Post-processes images from `torch.Tensor` to `numpy.ndarray`."""
        images = images.detach().cpu().numpy()
        images = (images + 1) * 255 / 2
        images = np.clip(images + 0.5, 0, 255).astype(np.uint8)
        images = images.transpose(0, 2, 3, 1)
        return images
    
direction = 1
shift_r = 10
shift_count = 11

save_images(codes, shift_r, shift_count, direction, generator, root_deformator, save_path)

# Plot images

In [ ]:

num_images = 10
all_images = torch.FloatTensor().cuda()
for i in range(num_images):
    images = torch.load(visualisation_data_path + '/images/images_new_direction_'+str(direction) + '/image_'+str(i)+'.pkl')
    image = torch.cat((images[0], images[1])).detach()
    all_images = torch.cat((all_images, image))
    del image, images
    
import matplotlib.pyplot as plt
import torchvision
plt.figure(figsize= (60,60))
grid = torchvision.utils.make_grid(all_images.clamp(min=-1, max=1),nrow=2, scale_each=True, normalize=True)
plt.imshow(grid.permute(1, 2, 0).cpu().numpy())

In [35]:
# z = codes[2].view(-1,512)
# direction = 70
# cf_image_path = 'images_cf/'
# ours_image_path = 'images_ours/'
# save_images(z, shifts_r, shifts_count, direction, G, deformator_cf, cf_image_path)
# save_images(z, shifts_r, shifts_count, direction, G, deformator_ours, ours_image_path)

54726729


## Plot images

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

images = []
for img_path in glob.glob('images/*.png'):
    images.append(mpimg.imread(img_path))

plt.figure(figsize=(20,10))
columns = 10
for i, image in enumerate(images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.imshow(image)